In [1]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, explode, to_timestamp, to_date, current_timestamp, when, lit

today = datetime.now().strftime("%Y/%m/%d")
BRONZE_PATH = f"s3a://bronze/posicao/{today}/"
SILVER_PATH = "s3a://silver/posicao/"

print(f"Lendo Bronze de: {BRONZE_PATH}")
print(f"Gravando Silver em: {SILVER_PATH}")

Lendo Bronze de: s3a://bronze/posicao/2025/11/03/
Gravando Silver em: s3a://silver/posicao/


In [2]:
spark = (
    SparkSession.builder.appName("BronzeToSilver_Parquet")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", True)
    .getOrCreate()
)

In [3]:
# Definir schema explícito para evitar erro de inferência

schema = StructType([
    StructField("hr", StringType(), True),
    StructField("l", ArrayType(
        StructType([
            StructField("c", StringType(), True),   # código visível (ex: "6L10-10")
            StructField("cl", IntegerType(), True), # código de linha usado pra outras chamadas de API
            StructField("sl", IntegerType(), True), # sentido (1 = ida, 2 = volta)
            StructField("lt0", StringType(), True), # terminal inicial
            StructField("lt1", StringType(), True), # terminal final
            StructField("qv", IntegerType(), True), # quantidade de veículos
            StructField("vs", ArrayType(
                StructType([
                    StructField("p", IntegerType(), True),  # código do veículo
                    StructField("a", BooleanType(), True),  # acessibilidade
                    StructField("ta", StringType(), True),  # timestamp de atualização
                    StructField("py", DoubleType(), True),  # latitude
                    StructField("px", DoubleType(), True),  # longitude
                ])
            ), True)
        ])
    ), True)
])

In [4]:
df_raw = spark.read.option("multiline", True).schema(schema).json(BRONZE_PATH)
df_raw.printSchema()

root
 |-- hr: string (nullable = true)
 |-- l: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- c: string (nullable = true)
 |    |    |-- cl: integer (nullable = true)
 |    |    |-- sl: integer (nullable = true)
 |    |    |-- lt0: string (nullable = true)
 |    |    |-- lt1: string (nullable = true)
 |    |    |-- qv: integer (nullable = true)
 |    |    |-- vs: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- p: integer (nullable = true)
 |    |    |    |    |-- a: boolean (nullable = true)
 |    |    |    |    |-- ta: string (nullable = true)
 |    |    |    |    |-- py: double (nullable = true)
 |    |    |    |    |-- px: double (nullable = true)



In [5]:
# Explosão, limpeza e classificação

df_exploded = (
    df_raw
    .withColumn("linha", explode(col("l")))
    .withColumn("veiculo", explode(col("linha.vs")))
    .filter(col("linha.cl").isNotNull())
)

# Classificação entre linhas regulares e técnicas (ex: GUIN-10, TESTE)
df_exploded = df_exploded.withColumn(
    "tipo_linha",
    when((col("linha.cl") < 1000) | (col("linha.c").rlike("GUIN|TEST|TST")), lit("tecnica"))
    .otherwise(lit("regular"))
)

# Se quiser excluir as técnicas do Silver:
df_exploded = df_exploded.filter(col("tipo_linha") == "regular")

In [6]:
# Enriquecimento
df_clean = (
    df_exploded.select(
        col("linha.c").alias("codigo_linha_texto"),
        col("linha.cl").cast("int").alias("codigo_linha"),
        col("tipo_linha"),
        col("linha.sl").alias("sentido"),
        col("linha.lt0").alias("terminal_inicial"),
        col("linha.lt1").alias("terminal_final"),
        col("veiculo.p").alias("codigo_veiculo"),
        col("veiculo.a").alias("acessibilidade"),
        to_timestamp(col("veiculo.ta")).alias("ultima_atualizacao"),
        col("veiculo.py").alias("latitude"),
        col("veiculo.px").alias("longitude"),
        to_timestamp(col("hr")).alias("hora_referencia"),
    )
    .dropDuplicates(["codigo_veiculo", "hora_referencia"])
    .withColumn("data_ref", to_date(col("ultima_atualizacao")))
    .withColumn("data_coleta", to_date(col("hora_referencia")))
    .withColumn("data_ingestao", to_date(current_timestamp()))
    .withColumn("ingest_timestamp", current_timestamp())
)

In [7]:
# Salvar em Parquet com append 

(
    df_clean
    .write
    .format("parquet")
    .mode("append")
    .partitionBy("codigo_linha", "data_ref")
    .save(SILVER_PATH)
)

print("✅ Transformação concluída e salva em Parquet na camada Silver.")

✅ Transformação concluída e salva em Parquet na camada Silver.


In [8]:
df_result = spark.read.parquet(SILVER_PATH)
df_result.select("codigo_linha", "data_ref", "hora_referencia", "codigo_veiculo").show(10, truncate=False)

+------------+----------+-------------------+--------------+
|codigo_linha|data_ref  |hora_referencia    |codigo_veiculo|
+------------+----------+-------------------+--------------+
|32975       |2025-11-03|2025-11-03 00:08:00|31111         |
|32975       |2025-11-03|2025-11-03 00:16:00|31111         |
|32975       |2025-11-03|2025-11-03 23:54:00|31111         |
|32975       |2025-11-03|2025-11-03 00:08:00|31117         |
|32975       |2025-11-03|2025-11-03 00:16:00|31117         |
|32975       |2025-11-03|2025-11-03 23:54:00|31117         |
|32975       |2025-11-03|2025-11-03 00:08:00|31171         |
|32975       |2025-11-03|2025-11-03 00:16:00|31171         |
|32975       |2025-11-03|2025-11-03 23:54:00|31171         |
|32975       |2025-11-03|2025-11-03 00:08:00|31186         |
+------------+----------+-------------------+--------------+
only showing top 10 rows

